In [26]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.base import clone
import re

import optuna
from optuna.samplers import TPESampler

from sklearn.model_selection import *
from sklearn.preprocessing import *

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

from sklearn.metrics import *

pd.set_option('display.max_columns', None)
from IPython.display import clear_output
from tqdm import tqdm, trange
from tabulate import tabulate
import random
import time
import logging
from IPython.display import display
from IPython.display import display, HTML
from colorama import Fore
from datetime import datetime
from sklearn.ensemble import *

In [27]:
%%time

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')



def update(df):
    
    t = 100
    
    df['accident'] = df['accident'].map({
        'None reported': 'not_reported',
        'At least 1 accident or damage reported': 'reported'
    })
    df['transmission'] = df['transmission'].str.replace('/', '').str.replace('-', '')
    df['transmission'] = df['transmission'].str.replace(' ', '_')
    
    cat_c = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title']
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for col in re_:
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
        
    for col in cat_c:
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')
        
    return df

train  = update(train)
test   = update(test)

CPU times: user 707 ms, sys: 81.6 ms, total: 789 ms
Wall time: 794 ms


In [28]:
%%time

def feature(df):
    current_year = datetime.now().year

    df['Vehicle_Age'] = current_year - df['model_year']

    df['Mileage_per_Year'] = df['milage'] / df['Vehicle_Age']

    def extract_horsepower(engine):
        try:
            return float(engine.split('HP')[0])
        except:
            return None

    def extract_engine_size(engine):
        try:
            return float(engine.split(' ')[1].replace('L', ''))
        except:
            return None

    df['Horsepower'] = df['engine'].apply(extract_horsepower)
    df['Engine_Size'] = df['engine'].apply(extract_engine_size)
    df['Power_to_Weight_Ratio'] = df['Horsepower'] / df['Engine_Size']

    luxury_brands =  ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land', 
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini', 
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']
    df['Is_Luxury_Brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

    df['Accident_Impact'] = df.apply(lambda x: 1 if x['accident'] == 1 and x['clean_title'] == 0 else 0, axis=1)
    
    return df

train = feature(train)
test = feature(test)

CPU times: user 870 ms, sys: 33.8 ms, total: 904 ms
Wall time: 906 ms


In [29]:
train.drop_duplicates(inplace = True)

In [30]:
train.shape

(192542, 21)

In [31]:
train.head()

,Unnamed: 0,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Vehicle_Age,Mileage_per_Year,Horsepower,Engine_Size,Power_to_Weight_Ratio,Is_Luxury_Brand,Accident_Impact
0,0,0.0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,AT,Yellow,Gray,missing,Yes,4200,17,12529.411765,172.0,1.6,107.500000,0,0
1,1,1.0,Lincoln,noise,2002,143250,Gasoline,noise,AT,Silver,Beige,missing,Yes,4999,22,6511.363636,NaN,NaN,NaN,0,0
2,2,2.0,Chevrolet,noise,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,AT,Blue,Gray,missing,Yes,13900,22,6215.045455,320.0,5.3,60.377358,0,0
3,3,3.0,Genesis,noise,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission_wDual_Shift_Mode,Black,Black,missing,Yes,45000,7,2785.714286,420.0,5.0,84.000000,0,0
4,4,4.0,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7Speed_AT,Black,Beige,missing,Yes,97500,3,2462.666667,208.0,2.0,104.000000,1,0


In [38]:
train.drop(columns = ['Unnamed: 0', 'id'], inplace = True)

KeyError: "['Unnamed: 0'] not found in axis"

In [33]:
test.drop(columns = ['Unnamed: 0', 'id'], inplace = True)

In [34]:
%%time

X = train.drop(['price'], axis=1)
y = train['price']
cat_features = X.select_dtypes(include=['category']).columns.tolist()

SEED = 601
n_splits = 5

def Train_ML(X, y, model, test, n_splits=n_splits):
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    rmse_scores = []
    test_preds = np.zeros((test.shape[0], n_splits)) 
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        val_predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
        rmse_scores.append(rmse)
        
        test_preds[:, fold] = model.predict(test)
    
    avg_rmse = np.mean(rmse_scores)
    
    mean_test_preds = np.mean(test_preds, axis=1)
    
    header = f"\n{'Final Validation RMSE:':<25} {avg_rmse:.5f}\n"
    print(header)
    
    return mean_test_preds

lgb_params ={'learning_rate': 0.017521301504983752, 'max_depth': 42, 'reg_alpha': 0.06876635751774487, 
 'reg_lambda': 9.738899198284985, 'num_leaves': 131, 'subsample': 0.2683765421728044, 
 'colsample_bytree': 0.44346036599709887} 

params1 = {'learning_rate': 0.015387355282525047, 'num_leaves': 287, 'max_depth': 10, 'min_child_samples': 32,
 'subsample': 0.5678602068076838, 'colsample_bytree': 0.5254867750210618, 'reg_alpha': 8.515713311140541e-05,
 'reg_lambda': 9.929128235845939, 'scale_pos_weight': 1.031529653438031, 'max_bin': 2894,
 'min_split_gain': 8.135732868325528e-05, 'min_child_weight': 0.9684228603448732,'boosting_type': 'gbdt',
 'objective': 'regression','metric': 'rmse'}

params2 = {'learning_rate': 0.01383980028736371, 'num_leaves': 185, 'max_depth': 11,
'min_child_samples': 47, 'subsample': 0.5179868722209913, 'colsample_bytree': 0.5099485937874763,
'reg_alpha': 2.6213681351209853e-08, 'reg_lambda': 5.24590000227586, 'scale_pos_weight': 3.6179756319298417, 
'max_bin': 2910, 'min_split_gain': 0.007117579745695504, 'min_child_weight': 0.21739378651138622}

lgb1 = LGBMRegressor(**lgb_params, random_state=SEED, verbose=-1, n_estimators=200)
lgb2 = LGBMRegressor(**params1, random_state=SEED, verbose=-1, n_estimators=200)
lgb3 = LGBMRegressor(**params2, random_state=SEED, verbose=-1, n_estimators=200)


estimator1 = VotingRegressor([
    ('LGBM_Tunned', lgb1), 
    ('LGBM_Tunned_1', lgb2), 
    ('LGBM_Tunned_2', lgb3)
], weights=[0.2, 0.7, 0.1])

vmp = Train_ML(X, y, estimator1, test)


Final Validation RMSE:    72475.84024

CPU times: user 2min 17s, sys: 51.5 s, total: 3min 9s
Wall time: 1min 32s


In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df):
    df_processed = df.copy()
    categorical_columns = df_processed.select_dtypes(include=['object', 'category']).columns
    
    le = LabelEncoder()
    
    for col in categorical_columns:
        df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    
    return df_processed

def Train_ML(X, y, model, test, n_splits=5):
    SEED = 601
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    rmse_scores = []
    test_preds = np.zeros((test.shape[0], n_splits))
    
    # Preprocess X and test data
    X_processed = preprocess_data(X)
    test_processed = preprocess_data(test)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_processed)):
        X_train, X_val = X_processed.iloc[train_idx], X_processed.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        val_predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
        rmse_scores.append(rmse)
        
        test_preds[:, fold] = model.predict(test_processed)
    
    avg_rmse = np.mean(rmse_scores)
    
    mean_test_preds = np.mean(test_preds, axis=1)
    
    header = f"\n{'Final Validation RMSE:':<25} {avg_rmse:.5f}\n"
    print(header)
    
    return mean_test_preds

# Assume train and test are your DataFrames
X = train.drop(['price'], axis=1)
y = train['price']

lgb_params = {'learning_rate': 0.017521301504983752, 'max_depth': 42, 'reg_alpha': 0.06876635751774487, 
              'reg_lambda': 9.738899198284985, 'num_leaves': 131, 'subsample': 0.2683765421728044, 
              'colsample_bytree': 0.44346036599709887}

params1 = {'learning_rate': 0.015387355282525047, 'num_leaves': 287, 'max_depth': 10, 'min_child_samples': 32,
           'subsample': 0.5678602068076838, 'colsample_bytree': 0.5254867750210618, 'reg_alpha': 8.515713311140541e-05,
           'reg_lambda': 9.929128235845939, 'scale_pos_weight': 1.031529653438031, 'max_bin': 2894,
           'min_split_gain': 8.135732868325528e-05, 'min_child_weight': 0.9684228603448732, 'boosting_type': 'gbdt',
           'objective': 'regression', 'metric': 'rmse'}

params2 = {'learning_rate': 0.01383980028736371, 'num_leaves': 185, 'max_depth': 11,
           'min_child_samples': 47, 'subsample': 0.5179868722209913, 'colsample_bytree': 0.5099485937874763,
           'reg_alpha': 2.6213681351209853e-08, 'reg_lambda': 5.24590000227586, 'scale_pos_weight': 3.6179756319298417, 
           'max_bin': 2910, 'min_split_gain': 0.007117579745695504, 'min_child_weight': 0.21739378651138622}

lgb1 = LGBMRegressor(**lgb_params, random_state=SEED, verbose=-1, n_estimators=200,n_jobs=-1)
lgb2 = LGBMRegressor(**params1, random_state=SEED, verbose=-1, n_estimators=200,n_jobs=-1)
lgb3 = LGBMRegressor(**params2, random_state=SEED, verbose=-1, n_estimators=200,n_jobs=-1)

estimator1 = VotingRegressor([
    ('LGBM_Tunned', lgb1), 
    ('LGBM_Tunned_1', lgb2), 
    ('LGBM_Tunned_2', lgb3)
], weights=[0.2, 0.7, 0.1])

vmp = Train_ML(X, y, estimator1, test)


Final Validation RMSE:    72489.43735



In [1]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
y_predict_train = estimator1.predict(x_train)

In [36]:
X.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'Vehicle_Age', 'Mileage_per_Year', 'Horsepower', 'Engine_Size',
       'Power_to_Weight_Ratio', 'Is_Luxury_Brand', 'Accident_Impact'],
      dtype='object')

In [22]:
import joblib as jo
jo.dump(estimator1,'usa_model.joblib')

['usa_model.joblib']